## **Agents**

An agent-based workflow where LLMs act autonomously within a loop, interacting with their environment and receiving feedback to refine their actions and decisions.

<img src="https://raw.githubusercontent.com/BrightPool/udemy-prompt-engineering-course/refs/heads/main/images/autonomous-agent.webp" alt="Agents" width="500">


## **Use Cases:**

- Building a personal research assistant that autonomously searches academic papers, extracts key findings, and generates literature review summaries based on specific research questions.
- Creating an autonomous code reviewer that analyzes pull requests, identifies potential bugs and security issues, suggests improvements, and generates detailed review comments.
- Developing a customer support agent that handles inquiries by searching knowledge bases, generating appropriate responses, and escalating complex issues to human agents when needed.
- Managing social media presence by analyzing trending topics, generating relevant content, scheduling posts, and engaging with followers through personalized responses.
- Building an autonomous testing agent that generates test cases, executes tests, analyzes failures, and provides detailed bug reports with suggested fixes.
- Creating a data monitoring agent that continuously analyzes system metrics, detects anomalies, investigates root causes, and generates incident reports with recommended actions.

In [ ]:
%pip install openai pydantic --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [7]:
import json
import os
from dotenv import load_dotenv
from openai import OpenAI

In [9]:
load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
MODEL="gpt-4o"

In [10]:
# Here we define a sample function the agent can call.
# In practice, you might connect to real APIs or services for your agent's tasks.
def search_knowledge_base(query: str) -> str:
    """
    Pretend to look for information about the provided query in a knowledge base.
    Returns a brief summary as a string.
    """
    # For demonstration, we'll return a placeholder string.
    # In a real scenario, you'd perform a search and generate a proper result.
    return f"Summary for '{query}': This is a simulated summary from the knowledge base."

def generate_keywords(query: str) -> str:
    """
    Pretend to generate keywords for the provided query.
    Returns a comma-separated string of keywords.
    """
    return f" The keywords for the '{query}': This has a list of 3 keywords from the knowledge base."

# Next, we define our function schema to provide to the model.
tools = [{
    "type": "function",
    "function": {
        "name": "search_knowledge_base",
        "description": "Query a knowledge base to retrieve relevant info on a topic.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The user question or search query."
                },
            },
            "required": [
                "query",
            ],
            "additionalProperties": False
        },
        "strict": True
    }
},{
    "type": "function",
    "function": {
        "name": "generate_keywords",
        "description": "Query a knowledge base to retrieve relevant keywords about a topic.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The user question or search query."
                },
            },
            "required": [
                "query",
            ],
            "additionalProperties": False
        },
        "strict": True
    }
  }
]

# We simulate a user asking the agent a question that might prompt the model to call the function.
messages = [
    {
        "role": "user",
        "content": "Hello, I'd like to know more about quantum computing."
    }
]

# Step 1: We call the model, providing the function tools. The model may decide to call the tool.
completion_1 = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools
)

In [11]:
# Step 2: The model might return a tool call. We'll check for any function calls and execute them.
tool_calls = completion_1.choices[0].message.tool_calls

results = []
if tool_calls:
    for call in tool_calls:
        function_name = call.function.name
        # Parse the JSON arguments:
        function_args = json.loads(call.function.arguments)

        if function_name == "search_knowledge_base":
            print('Calling function - ', function_name)
            search_result = search_knowledge_base(**function_args)
            # We'll store the result and associate it with this tool_call
            results.append(
                {
                    "tool_call_id": call.id,
                    "content": search_result
                }
            )
        if function_name ==  "generate_keywords":
            print('Calling function - ', function_name)
            keyword_result = generate_keywords(**function_args)
            # We'll store the result and associate it with this tool_call
            results.append(
                {
                    "tool_call_id": call.id,
                    "content": keyword_result
                }
            )

Calling function -  search_knowledge_base


In [12]:
completion_1.choices[0].message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_nO13904J22RpQBuabmtc6QX1', function=Function(arguments='{"query":"quantum computing"}', name='search_knowledge_base'), type='function')])

In [19]:
# Step 3: We send the results back to the model so it can incorporate them into its final answer.
# We'll add both the original function call message and the tool's reply to our conversation.

if tool_calls and results:
    # Append the tool call message from the model
    messages.append(completion_1.choices[0].message)
    # Append our result(s) as 'tool' role messages
    for r in results:
        messages.append({
            "role": "tool",
            "tool_call_id": r["tool_call_id"],
            "content": r["content"]
        })
messages

[{'role': 'user',
  'content': "Hello, I'd like to know more about quantum computing."},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_fifbg1ofPCEB4RGzL1Ha1VkQ', function=Function(arguments='{"query":"quantum computing"}', name='search_knowledge_base'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_fifbg1ofPCEB4RGzL1Ha1VkQ',
  'content': 'Here is a summary of the topic you asked for quantum computing : \n'}]

In [20]:
# Step 4: Make another call to the model, now that it has the tool's output.
completion_2 = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools
)

# Finally, we print out the model's final answer. This is how the user sees it.
final_answer = completion_2.choices[0].message.content
print("Final Answer from Agent:")
print(final_answer)

Final Answer from Agent:
Quantum computing is an advanced field of computing based on the principles of quantum mechanics, a fundamental theory in physics that describes the nature of matter and energy on an atomic and subatomic level. Unlike classical computers, which use bits as the smallest unit of data and operate on binary states (0 or 1), quantum computers utilize quantum bits, or qubits, which can exist in multiple states at once due to the phenomenon known as superposition.

**Key Concepts in Quantum Computing:**

1. **Superposition:** This property allows qubits to be in a combination of 0 and 1 states simultaneously. This means a quantum computer can process a vast number of possibilities at once, increasing its computational power exponentially for certain tasks.

2. **Entanglement:** Qubits can be entangled, meaning the state of one qubit is directly related to the state of another, no matter the distance between them. This property enables quantum computers to solve comple

------------------------------

## An Alternative Approach

If an agent has finished all of it's tool calls, then it has likely finished with the task.

Therefore we could also write the above code within a while loop that checks if the agent has finished all of it's tool calls. If it has, then we can break the loop.

In [16]:
def search_knowledge_base(query: str) -> str:
    """
    Pretend to look for information about the provided query in a knowledge base.
    """
    return f"Here is a summary of the topic you asked for {query} : \n"

def generate_keywords(query: str) -> str:
    """
    Pretend to generate keywords for the provided query.
    Returns a comma-separated string of keywords.
    """
    return f" The keywords for the '{query}': This has a list of 3 keywords from the knowledge base."

In [17]:
messages = [{"role": "user", "content": "Hello, I'd like to know more about quantum computing."}]

while True:
    completion = client.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    tool_calls = completion.choices[0].message.tool_calls

    if not tool_calls:
        break

    results = [] # Store results for this iteration
    for call in tool_calls:
        messages.append(completion.choices[0].message)

        function_name = call.function.name
        function_args = json.loads(call.function.arguments)

        if function_name == "search_knowledge_base":
            result = search_knowledge_base(**function_args)
        elif function_name == "generate_keywords":
            result = generate_keywords(**function_args)
        else:
            result = "Error: Unknown function"

        # Store the result with its corresponding tool_call_id
        results.append({"tool_call_id": call.id, "content": result})

    # Append the results to the messages list after processing all tool calls
    for r in results:
        messages.append({"role": "tool", "tool_call_id": r["tool_call_id"], "content": r["content"]})

In [18]:
final_answer = completion.choices[0].message.content
print("Final Answer from Agent:")
print(final_answer)

Final Answer from Agent:
Quantum computing is an advanced field of computing based on the principles of quantum mechanics, which is a fundamental theory in physics describing nature at the smallest scales, such as that of atoms and subatomic particles.

### Key Aspects of Quantum Computing:

1. **Quantum Bits (Qubits):** Unlike classical computers that use bits (0 or 1), quantum computers use quantum bits or qubits, which can exist in multiple states at once, thanks to superposition—a fundamental principle of quantum mechanics.

2. **Superposition:** This allows qubits to perform many calculations simultaneously, vastly increasing the computational power of quantum systems for certain tasks.

3. **Entanglement:** Qubits can be entangled, a peculiar quantum phenomenon where the state of one qubit directly affects the state of another, no matter the distance between them. This property is harnessed for complex problem-solving and secure communications.

4. **Quantum Gates and Circuits:**